In [ ]:
pip install paho-mqtt
pip install Flask

In [ ]:
import paho.mqtt.client as mqtt
import json
from flask import Flask, jsonify
from werkzeug.serving import run_simple
from threading import Thread

app = Flask(__name__)

# O caminho para o arquivo JSON
json_file_path = "mqtt_data.json"

# Dicionário para armazenar os dados
data_dict = {}

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    topics = ["/TEF/Vita300/attrs/user", "/TEF/Vita300/attrs/validade", "/TEF/Vita300/attrs/estoque"]
    
    for topic in topics:
        client.subscribe(topic)
        print(f"Subscribed to {topic}")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    global data_dict
    
    # Adiciona os dados ao dicionário usando o nome do tópico como chave
    data_dict[msg.topic] = str(msg.payload)
    
    # Escreve o dicionário atualizado no arquivo JSON
    with open(json_file_path, "w") as json_file:
        json.dump(data_dict, json_file, indent=2)
        print(f"Data written to {json_file_path}")

# Configurações do MQTT
mqtt_broker_address = "46.17.108.113"
mqtt_port = 1883

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker_address, mqtt_port, 60)

# Funções para obter dados da API
@app.route('/validade', methods=['GET'])
def get_validade():
    return jsonify({'validade': data_dict.get("/TEF/Vita300/attrs/validade", "")})

@app.route('/estoque', methods=['GET'])
def get_estoque():
    return jsonify({'estoque': data_dict.get("/TEF/Vita300/attrs/estoque", "")})

@app.route('/user', methods=['GET'])
def get_user():
    return jsonify({'user': data_dict.get("/TEF/Vita300/attrs/user", "")})

def flask_thread():
    run_simple('127.0.0.1', 5000, app, use_reloader=False)

if __name__ == '__main__':
    # Iniciar o Flask em uma thread separada
    flask_thread = Thread(target=flask_thread)
    flask_thread.start()

    # Iniciar o loop MQTT
    client.loop_forever()